In [1]:
import sys
sys.path.append("..")
import cv2
from ultralytics import YOLO
from tracker import *
import time

model = YOLO('yolov8n.pt')

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:  
        colorsBGR = [x, y]
        
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap = cv2.VideoCapture('micra50kmhup.mp4')
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 

count = 0
tracker = Tracker()

cy1 = 390  
cy2 = 440  
offset = 6
vh_up = {}
vh_up_speed = {}
counter1 = []
calibration_value = 0.6
speed_of_vehicle_22 = None  # Variable to store the speed of vehicle ID 22

while True:    
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    list = []
    if results:  
        for detection in results[0].boxes:
            xyxy = detection.xyxy[0].tolist()
            class_id = int(detection.cls[0].item())
            c = class_list[class_id]
            if 'car' in c:
                list.append(xyxy[:4])
        bbox_id = tracker.update(list)

        for bbox in bbox_id:
            if len(bbox) >= 5:
                x3, y3, x4, y4, id = bbox
                x3, y3, x4, y4 = map(int, bbox[:4])
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)
                cv2.putText(frame, str(id), (x3, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 2)

                cx = (x3 + x4) // 2
                cy = (y3 + y4) // 2

                if cy2 < (cy + offset) and cy2 > (cy - offset):
                    vh_up[id] = time.time()
                if id in vh_up and cy1 < (cy + offset) and cy1 > (cy - offset):
                    if id == 22:  # Only for vehicle ID 22
                        elapsed1_time = time.time() - vh_up[id]
                        if elapsed1_time > 0 and id not in counter1:
                            counter1.append(id)      
                            distance1 = 20 * calibration_value
                            a_speed_ms1 = distance1 / elapsed1_time
                            a_speed_kh1 = a_speed_ms1 * 3.6
                            vh_up_speed[id] = a_speed_kh1
                            speed_of_vehicle_22 = a_speed_kh1

    cv2.line(frame, (240, cy1), (1000, cy1), (255, 255, 255), 1)
    cv2.putText(frame, ('L1'), (277, 320), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
    cv2.line(frame, (300, cy2), (1100, cy2), (255, 255, 255), 1)
    cv2.putText(frame, ('L2'), (300, 367), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

# Print the speed of vehicle ID 22 at the end
if speed_of_vehicle_22 is not None:
    print(f"Final speed of vehicle ID 22 (Calibration Vehicle) : {speed_of_vehicle_22:.2f} Km/h")
else:
    print("Vehicle ID 22 (Calibration Vehicle) was not detected or did not cross the line.")


100%|██████████| 6.23M/6.23M [00:00<00:00, 7.50MB/s]


Vehicle ID 22 (Calibration Vehicle) was not detected or did not cross the line.
